# Supertrend Crypto Trading Bot

In [ ]:
# Average True Range (ATR): measure of market volatility via decomposing the entire range of an assest price for that period
# True Range (TR): Max [current high - current low] OR Max [abs(current high - previous close)] 
# OR Max [abs(current low - previous close)] i.e. Max[CH-CL, abs(CH-PC), abs(CL-PC)]
# ATR is just the average of the TR over a specified period
#
#
# code adapted from @Part Time Larry on YouTube
# https://github.com/hackingthemarkets

In [6]:
pip install ccxt

  Using cached yarl-1.6.3-cp38-cp38-win_amd64.whl (125 kB)
  Attempting uninstall: yarl
    Found existing installation: yarl 1.7.0
    Uninstalling yarl-1.7.0:
      Successfully uninstalled yarl-1.7.0
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install schedule

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install ta

  Created wheel for ta: filename=ta-0.7.0-py3-none-any.whl size=28718 sha256=42fabe0f21b723f1a2bfbce62f159f8f87f156905073fd9b46d2b0cb1bc4b399
  Stored in directory: c:\users\mathe\appdata\local\pip\cache\wheels\bb\7c\a0\9c72e50ddef1f7c3d9003bf4ccc5d5c8deb24828d4eb156fc8
Successfully built ta
Note: you may need to restart the kernel to use updated packages.


In [9]:
import ta # technical analysis library
import schedule
import ccxt
import pandas as pd
pd.set_option('display.max_rows', None) # displays all rows

import warnings
warnings.filterwarnings('ignore')

import numpy as np
from datetime import datetime
import time

In [ ]:
# change such that it connects to your key
NDAX_API_KEY = 'yourndaxapikey' # create a general key - read only
NDAX_SECRET_KEY = 'yourndaxsecretkey' # create a trading key

In [ ]:
# change based on site/broker used
ndax = ccxt.ndax({
    'apiKey': config.NDAX_API_KEY,
    'secret': config.NDAX_SECRET_KEY,
})

In [10]:
# True Range
def tr(data):
    data['previous_close'] = data['close'].shift(1)
    data['high-low'] = abs(data['high'] - data['low']) # current high - current low
    data['high-pc'] = abs(data['high'] - data['previous_close']) # current high - previous close
    data['low-pc'] = abs(data['low'] - data['previous_close']) # current low - precious close

    tr = data[['high-low', 'high-pc', 'low-pc']].max(axis=1)
    # Max[CH-CL, abs(CH-PC), abs(CL-PC)]
    
    return tr


In [12]:
# Average True Range
def atr(data, period):
    data['tr'] = tr(data)
    atr = data['tr'].rolling(period).mean() # finds the atr of the tr over a given period length

    return atr

In [13]:
# in an UPTREND whenever closing price closes above the upper band
# in a DOWNTREND whenever closing price closes below the lower band
def supertrend(df, period=7, atr_multiplier=3):
    hl2 = (df['high'] + df['low']) / 2
    df['atr'] = atr(df, period)
    df['upperband'] = hl2 + (atr_multiplier * df['atr']) 
    # basic upper band = ((high + low) / 2) + (multiplier * atr) 
    df['lowerband'] = hl2 - (atr_multiplier * df['atr']) 
    # basic lower band = ((high + low) / 2) - (multiplier * atr)
    df['in_uptrend'] = True

    for current in range(1, len(df.index)):
        previous = current - 1

        if df['close'][current] > df['upperband'][previous]: # UPTREND
            df['in_uptrend'][current] = True
        elif df['close'][current] < df['lowerband'][previous]: # DOWNTREND
            df['in_uptrend'][current] = False
        else:
            df['in_uptrend'][current] = df['in_uptrend'][previous] # in the same trend

            if df['in_uptrend'][current] and df['lowerband'][current] < df['lowerband'][previous]:
                df['lowerband'][current] = df['lowerband'][previous] # generates the flat portion of the lower band

            if not df['in_uptrend'][current] and df['upperband'][current] > df['upperband'][previous]:
                df['upperband'][current] = df['upperband'][previous] # generates the flat portion of the upper band
        
    return df

In [1]:
# can pre-set this to force action, can buy in account manually change to True so it will then automate after
in_position = False

In [ ]:
def check_buy_sell_signals(df):
    global in_position

    print("checking for buy and sell signals")
    print(df.tail(5))
    last_row_index = len(df.index) - 1
    previous_row_index = last_row_index - 1

    if not df['in_uptrend'][previous_row_index] and df['in_uptrend'][last_row_index]:
        print("changed to uptrend, buy") # buy on the uptrend
        if not in_position:
            order = kraken.create_market_buy_order('ETH/USD', 0.05)
            print(order)
            in_position = True
        else:
            print("already in position, nothing to do") # stay if in same trend
    
    if df['in_uptrend'][previous_row_index] and not df['in_uptrend'][last_row_index]:
        if in_position:
            print("changed to downtrend, sell") # sell on the downtrend
            order = kraken.create_market_sell_order('ETH/USD', 0.05)
            print(order)
            in_position = False
        else:
            print("You aren't in position, nothing to sell") # trend hasn't change to buy in yet

In [ ]:
def run_bot():
    print(f"Fetching new bars for {datetime.now().isoformat()}")
    bars = kraken.fetch_ohlcv('ETH/USDT', timeframe='1m', limit=100) # limit is the number of units for the timeframe
    df = pd.DataFrame(bars[:-1], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms') # makes it more human readable

    supertrend_data = supertrend(df)
    
    check_buy_sell_signals(supertrend_data)

In [ ]:
schedule.every(10).seconds.do(run_bot)

while True:
    schedule.run_pending()
    time.sleep(1)